**We use https://github.com/wwymak/avazu-kaggle to download and prepare our dataset.**

In [2]:
import pandas as pd
from numpy import loadtxt
import matplotlib.pyplot as plt
import torch
import pickle
import json
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

# from drive.MyDrive.RCSYS_finalproject.DeepCTRTorch.deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
# from drive.MyDrive.RCSYS_finalproject.DeepCTRTorch.deepctr_torch.models import *
# from drive.MyDrive.RCSYS_finalproject.DeepCTRTorch.deepctr_torch.models.SExdeepfm import *
# from drive.MyDrive.RCSYS_finalproject.DeepCTRTorch.deepctr_torch.inputs import VarLenSparseFeat

**Download avazu dataset (only one time)**

In [3]:
#first we instal kaggle to use Kaggle API
!pip install kaggle

In [4]:
#And upgrade the kaggle version
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 58 kB 3.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73051 sha256=bc8138bc31f77877ec4386bb0186035235394388b5736c6647342fc7bb7358bb
  Stored in directory: /root/.cache/pip/wheels/62/d6/58/5853130f941e75b2177d281eb7e44b4a98ed46dd155f556dc5
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [5]:
#using https://github.com/Kaggle/kaggle-api we download of kaggle.json, a file containing our API credentials
#And then we upload it to cola using https://colab.research.google.com/github/corrieann/kaggle/blob/master/kaggle_api_in_colab.ipynb#scrollTo=hMY4CFezjcG-
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 67 bytes


In [6]:
#download avazu dataset
!kaggle competitions download -c avazu-ctr-prediction

 99% 1.18G/1.19G [00:09<00:00, 139MB/s]
100% 1.19G/1.19G [00:09<00:00, 134MB/s]


In [7]:
#Then unzip the file of avazu dataset

!unzip /content/avazu-ctr-prediction.zip

Archive:  /content/avazu-ctr-prediction.zip
  inflating: sampleSubmission.gz     
  inflating: test.gz                 
  inflating: train.gz                


In [1]:
#read the file to pandas dataframe
import pandas as pd

rca_df = pd.read_csv('/content/train.gz')
rca_df.head()

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157


In [2]:
rca_df

,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,ddd2926e,44956a24,1,2,15706,320,50,1722,0,35,-1,79
1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,96809ac8,711ee120,1,0,15704,320,50,1722,0,35,100084,79
2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,b3cf8def,8a4875bd,1,0,15704,320,50,1722,0,35,100084,79
3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,e8275b8f,6332421a,1,0,15706,320,50,1722,0,35,100084,79
4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,7801e8d9,07d7df22,a99f214a,9644d0bf,779d90c2,1,0,18993,320,50,2161,0,35,-1,157
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40428962,9.998753e+18,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,dcf40646,d9a2d3a9,1,0,17262,320,50,1872,3,39,100173,23
40428963,9.999038e+18,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,e5751bb2,cd5b1559,1,2,23160,320,50,2667,0,47,-1,221
40428964,9.999585e+18,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,47e5bb5b,2cd8ff6d,1,0,20969,320,50,2372,0,813,-1,46
40428965,9.999636e+18,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,2347f47a,0f2161f8,454877d9,2dfaa2fb,f063b808,1,0,16859,320,50,1887,3,39,100194,23


In [3]:
rca_df['hour'].unique()

array([14102100, 14102101, 14102102, 14102103, 14102104, 14102105,
       14102106, 14102107, 14102108, 14102109, 14102110, 14102111,
       14102112, 14102113, 14102114, 14102115, 14102116, 14102117,
       14102118, 14102119, 14102120, 14102121, 14102122, 14102123,
       14102200, 14102201, 14102202, 14102203, 14102204, 14102205,
       14102206, 14102207, 14102208, 14102209, 14102210, 14102211,
       14102212, 14102213, 14102214, 14102215, 14102216, 14102217,
       14102218, 14102219, 14102220, 14102221, 14102222, 14102223,
       14102300, 14102301, 14102302, 14102303, 14102304, 14102305,
       14102306, 14102307, 14102308, 14102309, 14102310, 14102311,
       14102312, 14102313, 14102314, 14102315, 14102316, 14102317,
       14102318, 14102319, 14102320, 14102321, 14102322, 14102323,
       14102400, 14102401, 14102402, 14102403, 14102404, 14102405,
       14102406, 14102407, 14102408, 14102409, 14102410, 14102411,
       14102412, 14102413, 14102414, 14102415, 14102416, 14102

In [4]:
rca_df.columns

Index(['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21'],
      dtype='object')

In [5]:
rca_df.shape

(40428967, 24)

In [6]:
#ignore the id column and hour
df = rca_df[['click','hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']]

In [1]:
from google.colab import  drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#if we want to test on last rows, save the validation set into pickle file.
df_val = df[df.hour >= 14103000]
pd.to_pickle(df_val, '/content/drive/MyDrive/RCSYS - final project/avazu/val.pkl')

In [7]:
#separates out the training data into subsets by day and save it into pickle files.
file_num = 1
for i in range(14102200, 14103100, 100):
    df_sub = df[(df.hour < i) & (df.hour >= (i - 100))]
    fname = 'train{}.pkl'.format(file_num)
    pd.to_pickle(df_sub, fname)
    file_num +=1

**After we Finish the download we can load the dataset from the pickle files**

In [2]:
import pandas as pd

In [ ]:
#train1 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train1.pkl') #we used for dev test hyper-param opt.
train2 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train2.pkl')
train3 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train3.pkl')
train4 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train4.pkl')
train5 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train5.pkl')
train6 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train6.pkl')
train7 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train7.pkl')
train8 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train8.pkl')
train9 = pd.read_pickle('/content/drive/MyDrive/RCSYS - final project/avazu/train9.pkl')

In [ ]:
#If we use more then one file we can concat them to one df.
#train = pd.concat([train6,train7,train8,train9])
# train = pd.concat([train2,train3,train4,train5])

In [ ]:
#We use the column names used by deepctr_torch API
data2 = pd.read_csv('/content/drive/MyDrive/RCSYS_finalproject/DeepCTRTorch/examples/criteo_sample.txt')

In [ ]:
#preparing the dataset
data = train
data = data.drop(['hour'], axis=1)

column_list = ['label']
column_list.extend(list(data2.columns[14:35]))
old_columns = list(data.columns)
columns = dict(zip(old_columns, column_list))

data.rename(columns=columns, inplace=True)

In [ ]:
train = pd.concat([train_2to5,train_6to9])
train.to_pickle('/content/drive/MyDrive/RCSYS_finalproject/avazu/train_2to9.pkl')

**After we finish the first preparing**

In [3]:
train_2to9 = pd.read_pickle('/content/drive/MyDrive/RCSYS_finalproject/avazu/train_2to9.pkl')

In [6]:
val = pd.read_pickle('/content/drive/MyDrive/RCSYS_finalproject/avazu/val.pkl')

In [7]:
val

,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
36210029,0,14103000,1005,1,ec47f32a,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,f118cfc8,74b0fc2c,1,0,23536,320,50,2695,0,35,100019,51
36210030,0,14103000,1005,1,ec47f32a,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,273a495f,7abbbd5c,1,0,23539,320,50,2695,0,35,100020,51
36210031,0,14103000,1005,0,db3ea4cd,5359cafe,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,1505cc9a,cdf6ea96,1,0,17239,320,50,1973,3,39,100148,23
36210032,0,14103000,1005,1,5b4d2eda,16a36ef3,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,e320afe4,86030d43,1,0,19771,320,50,2227,0,935,-1,48
36210033,0,14103000,1005,0,2328ee8e,7804dea6,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,b32feedb,945dd5c4,1,0,23723,320,50,2716,3,47,-1,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40428962,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,dcf40646,d9a2d3a9,1,0,17262,320,50,1872,3,39,100173,23
40428963,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,9c13b419,2347f47a,f95efa07,a99f214a,e5751bb2,cd5b1559,1,2,23160,320,50,2667,0,47,-1,221
40428964,0,14103023,1005,1,f61eaaae,6b59f079,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,47e5bb5b,2cd8ff6d,1,0,20969,320,50,2372,0,813,-1,46
40428965,1,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,3c4b944d,2347f47a,0f2161f8,454877d9,2dfaa2fb,f063b808,1,0,16859,320,50,1887,3,39,100194,23


In [ ]:
#due to ram limit we dont use the val part.
#val = pd.read_pickle('/content/drive/MyDrive/RCSYS_finalproject/avazu/val.pkl')
#train = pd.concat([train_2to9,val])

In [4]:
data = train_2to9

In [5]:
data

,label,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21
4122995,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,91ba4960,ecb851b2,1,0,21731,320,50,2502,0,35,100083,221
4122996,0,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,a99f214a,97b6a2c0,22fb7d04,1,0,21189,320,50,2424,1,161,100193,71
4122997,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,41ba54fa,0eb711ec,1,0,21761,320,50,2502,0,35,100084,221
4122998,1,1005,0,85f751fd,c4e18dd6,50e219e0,e96166c7,2347f47a,0f2161f8,ed276be5,0398906d,1e5925f1,1,0,21611,320,50,2480,3,297,100111,61
4122999,0,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,a1c3ddad,c6263d8a,1,0,21758,320,50,2502,0,35,-1,221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36210024,0,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,0f2161f8,a99f214a,b65e16a3,edead9f4,1,0,20633,320,50,2374,3,39,-1,23
36210025,0,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,72797d6c,dcd7a69a,1,0,17747,320,50,1974,2,39,-1,33
36210026,0,1005,0,85f751fd,c4e18dd6,50e219e0,92f5800b,ae637522,0f2161f8,a99f214a,ece5609d,981edffc,1,3,21191,320,50,2424,1,161,100193,71
36210027,0,1005,1,ce3307ec,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,d90a7774,cad4c01d,1,0,23626,320,50,2707,3,937,100156,61


In [ ]:
sparse_features = ['C' + str(i) for i in range(1, 22)]
target = ['label']

#1.Label Encoding for sparse features,and do simple Transformation for dense features
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])
# mms = MinMaxScaler(feature_range=(0, 1))
# data[dense_features] = mms.fit_transform(data[dense_features])

# 2.count #unique features for each sparse field,and record dense feature field name
fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                          for feat in sparse_features]

dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(
    linear_feature_columns + dnn_feature_columns)

train, test = train_test_split(data, test_size = 0.2)

In [ ]:
train.to_pickle('/content/drive/MyDrive/RCSYS_finalproject/avazu/avazu_train.pkl')
test.to_pickle('/content/drive/MyDrive/RCSYS_finalproject/avazu/avazu_test.pkl')

In [ ]:
with open('/content/drive/MyDrive/RCSYS_finalproject/avazu/dnn_feature_columns_avazu.p', 'wb') as fp:
    pickle.dump(dnn_feature_columns, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/RCSYS_finalproject/avazu/dnn_feature_columns_avazu.p', 'rb') as fp:
    dnn_feature_columns = pickle.load(fp)

with open('/content/drive/MyDrive/RCSYS_finalproject/avazu/linear_feature_columns_avazu.p', 'wb') as fp:
    pickle.dump(linear_feature_columns, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/RCSYS_finalproject/avazu/linear_feature_columns_avazu.p', 'rb') as fp:
    linear_feature_columns = pickle.load(fp)

with open('/content/drive/MyDrive/RCSYS_finalproject/avazu/feature_names_avazu.p', 'wb') as fp:
    pickle.dump(feature_names, fp, protocol=pickle.HIGHEST_PROTOCOL)

with open('/content/drive/MyDrive/RCSYS_finalproject/avazu/feature_names_avazu.p', 'rb') as fp:
    feature_names = pickle.load(fp)

In [ ]:
train

,label,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21
33494378,0,2,1,4000,3694,24,7214,233,0,1465686,2185719,7808,1,0,19,3,2,6,2,4,109,7
31621604,0,2,1,4000,3694,24,7214,233,0,1465686,127383,4010,1,0,347,3,2,68,3,4,0,6
24568238,0,2,0,691,6561,4,7214,233,0,1465686,4478397,1613,1,0,344,3,2,84,3,4,0,6
18410350,0,2,0,571,6987,2,7214,233,0,1465686,169177,4990,1,0,273,3,2,56,0,2,65,21
27227868,0,1,0,382,1183,6,7214,233,0,2195821,786676,2537,0,0,271,3,2,56,0,2,64,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21344757,0,2,0,3859,4454,4,7214,233,0,1465686,3921252,6661,1,0,200,3,2,45,2,8,56,1
7463163,0,2,0,1695,4592,24,7214,233,0,1465686,3898987,7431,1,1,1027,3,2,205,0,2,86,40
33591468,0,2,1,4127,5702,24,7214,233,0,1465686,3549078,3089,1,0,2358,3,2,384,2,2,59,6
14259617,0,2,0,571,6987,2,7214,233,0,1465686,4193548,5923,1,0,1368,3,2,238,0,4,65,52


In [ ]:
test

,label,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21
29059057,0,2,0,2401,5716,6,3218,70,20,1465686,1260557,3783,1,1,2039,3,2,341,0,2,45,52
22240386,0,2,0,2798,4371,24,7214,233,0,1465686,2490769,5836,1,0,641,3,2,124,0,10,58,28
19209000,1,2,0,1683,3478,4,7214,233,0,1465686,5287895,7486,1,0,569,2,4,103,2,4,0,8
12305221,0,2,1,2434,4654,4,7214,233,0,1465686,3395355,3661,1,0,627,3,2,62,3,11,62,6
32203800,0,2,0,2401,5716,6,7791,261,4,1465686,4876453,4746,1,0,916,3,2,188,3,19,86,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33539706,0,2,0,2401,5716,6,7336,70,4,1465686,5085456,3415,1,0,2295,3,2,376,3,2,0,6
16424790,0,2,1,1749,688,24,7214,233,0,1465686,5325602,6571,1,0,325,3,2,74,0,27,57,37
26196483,0,1,0,1117,5195,6,7214,233,0,553175,660544,456,0,0,349,3,2,7,3,4,59,6
23373232,0,2,0,2401,5716,6,6281,349,28,1465686,4935149,1665,1,0,1538,3,2,282,1,11,6,20
